# Warehouse Optimization

## Initilization

In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
data = pd.read_csv('ML_datafile.csv')
data.reset_index(drop=True, inplace=True)
data = data.drop(['Item No', 'Manufacturer No', 'Product Name', 'Date', 'STOCK TYPE', 'In Stock', 'List No',
                 'UPC Code', 'SaleRate'], axis=1)
data.dtypes
data.head(10)

,Manufacturer,Category,Category Path 1,Category Path 2,Category Path 3,Price(USD),Cost price,Inventory,Shipping,Profit,weight(lb),SaleRateNew
0,Supermicro,Accessory,Hardware & Software,Server & Workstation,Accessory,2.05,1.0000,57,0.0,1.0500,0.90,1.000000
1,APC,Surge Protection,Component,Power Protection / UPS,Surge Protection,11.07,8.3025,51,2.5,0.2675,1.33,1.457143
2,Vantec,120mm,Component,Fan & Heatsink,Case Fan,11.02,8.2650,19,2.5,0.2550,0.54,1.000000
3,Scythe,120mm,Component,Fan & Heatsink,Case Fan,11.02,8.2650,19,2.5,0.2550,0.50,1.117647
4,iMicro,Keyboard,Component,Input & Output Device,Keyboard,10.76,8.0700,26,2.5,0.1900,1.20,0.714286
5,be quiet!,120mm,Component,Fan & Heatsink,Case Fan,10.76,8.0700,79,2.5,0.1900,0.34,1.000000
6,Supermicro,Accessory,Component,Motherboard,Accessory,10.76,8.0700,19,2.5,0.1900,1.00,0.055556
7,Thermaltake,LGA 1155,Component,Fan & Heatsink,CPU Fan & Heatsink,10.76,8.0700,51,2.5,0.1900,0.70,1.159091
8,VCOM,DVI Cable,Component,Cable,DVI Cable,10.76,8.0700,42,2.5,0.1900,0.11,1.400000
9,VCOM,USB Cable,Component,Cable,USB Cable,10.76,8.0700,52,2.5,0.1900,0.30,1.238095


In [2]:
import pandas as pd
#importing plotly and cufflinks in offline mode
import cufflinks as cf

ModuleNotFoundError: No module named 'cufflinks'

## Exploratory Analysis

In [ ]:
# get some basic ideas about the data
def summerize_data(df, columns):
    for column in columns:
        print(column)
        if df.dtypes[column] == np.object: # Categorical data
            print(df[column].value_counts())
        else:
            print(df[column].describe())
            
        print('\n')
    
columns = ["Price(USD)", "Inventory", "weight(lb)", "SaleRateNew"]
summerize_data(data, columns)

# Missing Data Points Per Feature

In [ ]:
data.isnull().sum()

## Graphs

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm
# if using a Jupyter notebook, inlcude:
%matplotlib inline

In [ ]:
data["SaleRateNew"].plot.kde(bw_method=0.3)

In [ ]:
data["SaleRateNew"].plot.kde(bw_method=0.1)

In [ ]:
data["Price(USD)"].plot.kde(bw_method=0.3)

In [ ]:
data["Inventory"].plot.kde(bw_method=0.3)

In [ ]:
data[ "weight(lb)"].plot.kde(bw_method=0.3)

## corelation matrix

In [ ]:
#correlation matrix
import seaborn as sns
corrmat = data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);
corrmat

# Relationship with categorical values

In [ ]:
#box plot overallqual/saleprice
box_plot_df = data.sample(n=100, random_state=1)
var = 'Price(USD)'
box_plot_df = pd.concat([box_plot_df['SaleRateNew'], box_plot_df[var]], axis=1)
f, ax = plt.subplots(figsize=(20, 18))
fig = sns.boxplot(x=var, y="SaleRateNew", data=box_plot_df)
fig.axis(ymin=0, ymax=1);

In [ ]:
#import pandas_profiling as pp

#check profile report of the dataset

#profile = data.profile_report(title='Pandas Profiling Report')
profile = pandas_profiling.ProfileReport(data)
profile.to_file("EDA.html")

In [ ]:
X = data.drop('SaleRateNew', axis=1)
y = data['SaleRateNew']

In [ ]:
len(data['Category'].unique())

In [ ]:
data.head(5)

In [ ]:
data.columns

In [ ]:
# limit to categorical data using df.select_dtypes()
#nominal_data = data.drop(['Price(USD)', 'Cost price', 'Inventory', 'Shipping', 'Profit', 'weight(lb)', 'SaleRateNew'], axis=1)
#ordinal_data = data.drop(nominal_data.columns)
nominal_data = data.select_dtypes(include=[object])
ordinal_data = data.drop(list(nominal_data.columns), axis=1)
nominal_data = nominal_data.replace(np.nan, 'Default', regex=True)
nominal_data.head(30)

## Lable Encoding

In [ ]:
from sklearn import preprocessing
nominal_data.reset_index(drop=True, inplace=True)
le = preprocessing.LabelEncoder()
nominal_data_2 = nominal_data.apply(le.fit_transform)
nominal_data_2.head(10)

## One-Hot Encoding

In [ ]:
# TODO: create a OneHotEncoder object, and fit it to all of X

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()
# 2. FIT
enc.fit(nominal_data_2)
# 3. Transform
onehotlabels = enc.transform(nominal_data_2).toarray()
print(onehotlabels.shape)
# as you can see, you've the same number of rows 891
# but now you've so many more columns due to how we changed all the categorical data into numerical data
num_col = (len(nominal_data_2['Manufacturer'].unique())+
              len(nominal_data_2['Category'].unique())+
              len(nominal_data_2['Category Path 1'].unique())+
              len(nominal_data_2['Category Path 2'].unique())+
              len(nominal_data_2['Category Path 3'].unique()))
print(num_col)

In [ ]:
onehotlabels.shape

In [ ]:
encoded_data = pd.concat([ordinal_data, pd.DataFrame(onehotlabels)], axis=1)
print(encoded_data.shape)
print(encoded_data.head(5))

## Data Split

In [ ]:
X = encoded_data.drop('SaleRateNew', axis=1)
Y = encoded_data['SaleRateNew']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

## Model Creation (Linear Regression)

### Training

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)
model.fit(X_train, Y_train)

### Prediction

In [ ]:
Y_pred = model.predict(X_test)

### Scoring

In [ ]:
#import sklearn
#sklearn.metrics.precision_recall_fscore_support(Y_test, Y_pred)
model.score(X_test, Y_test)

## Model Creation (Lasso Regression)

### Training

In [ ]:
from sklearn import linear_model
model = linear_model.Lasso(alpha=0.01, max_iter=10000, normalize=False)
model.fit(X_train, Y_train)

### Prediction

In [ ]:
Y_pred = model.predict(X_test)

### Scoring

In [ ]:
#import sklearn
#sklearn.metrics.precision_recall_fscore_support(Y_test, Y_pred)
model.score(X_test, Y_test)